In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [40]:
df = pd.read_csv(r'C:\Users\nguyen\OneDrive\Tài liệu\baitaplonn\Time_Series_with_Python\notebook\data\raw\sales_train_validation.csv')
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


```
📊 Dữ liệu này nói về gì?

Đây là dữ liệu bán lẻ theo ngày (daily sales data) của chuỗi siêu thị Walmart ở Mỹ.
Nó ghi lại số lượng sản phẩm được bán mỗi ngày trong hơn 5 năm (2011–2016).
```

```
id: Mã định danh của sản phẩm, kết hợp item_id, store_id, và hậu tố _validation.
item_id: Mã sản phẩm.
dept_id: Mã phòng ban (department) chứa sản phẩm.
cat_id: Mã danh mục (category) chứa phòng ban.
store_id: Mã cửa hàng bán sản phẩm.
state_id: Bang/tiểu bang của cửa hàng.

Từ d_1 → d_1913 = doanh số (số lượng bán ra) theo từng ngày.
d_1 = 2011-01-29,
d_1913 = 2016-05-22 (mapping trong file calendar.csv).


```

# thông tin chung về dữ liệu 

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


```
-DataFrame có 30,490 dòng, đánh số từ 0 đến 30,489.
-Có 1,919 cột, cột đầu tiên là id, cột cuối cùng là d_1913.
- 1,913 cột kiểu số nguyên (int64),
-6 cột kiểu chuỗi / hỗn hợp (object).
DataFrame chiếm ~446 MB trong RAM.
```

# lọc gia trị missing

In [42]:
print(df.isnull().sum())
print(df.isnull().mean() * 100)

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
           ..
d_1909      0
d_1910      0
d_1911      0
d_1912      0
d_1913      0
Length: 1919, dtype: int64
id          0.0
item_id     0.0
dept_id     0.0
cat_id      0.0
store_id    0.0
           ... 
d_1909      0.0
d_1910      0.0
d_1911      0.0
d_1912      0.0
d_1913      0.0
Length: 1919, dtype: float64


=> tập dữ liệu không có giá trị missing

# kiểm tra giá trị ngoại lệ và bất thường

In [43]:
# Chọn các cột số
num_cols = df.select_dtypes(include=["int64", "float64"]).columns

outliers = {}

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Lọc các giá trị ngoại lệ
    mask = (df[col] < lower) | (df[col] > upper)
    outliers[col] = df.loc[mask, col]

# Ví dụ: xem số lượng ngoại lệ từng cột
for col, vals in outliers.items():
    print(f"{col}: {len(vals)} ngoại lệ")

d_1: 6979 ngoại lệ
d_2: 6880 ngoại lệ
d_3: 5990 ngoại lệ
d_4: 5972 ngoại lệ
d_5: 5068 ngoại lệ
d_6: 6361 ngoại lệ
d_7: 6241 ngoại lệ
d_8: 7152 ngoại lệ
d_9: 6707 ngoại lệ
d_10: 6129 ngoại lệ
d_11: 6086 ngoại lệ
d_12: 6000 ngoại lệ
d_13: 6126 ngoại lệ
d_14: 6468 ngoại lệ
d_15: 7247 ngoại lệ
d_16: 7311 ngoại lệ
d_17: 5925 ngoại lệ
d_18: 6079 ngoại lệ
d_19: 6018 ngoại lệ
d_20: 5950 ngoại lệ
d_21: 6470 ngoại lệ
d_22: 7236 ngoại lệ
d_23: 6891 ngoại lệ
d_24: 6354 ngoại lệ
d_25: 5960 ngoại lệ
d_26: 5975 ngoại lệ
d_27: 5953 ngoại lệ
d_28: 6366 ngoại lệ
d_29: 7069 ngoại lệ
d_30: 6926 ngoại lệ
d_31: 6055 ngoại lệ
d_32: 6228 ngoại lệ
d_33: 6058 ngoại lệ
d_34: 6174 ngoại lệ
d_35: 6198 ngoại lệ
d_36: 7056 ngoại lệ
d_37: 7330 ngoại lệ
d_38: 6449 ngoại lệ
d_39: 6088 ngoại lệ
d_40: 6038 ngoại lệ
d_41: 6208 ngoại lệ
d_42: 6501 ngoại lệ
d_43: 7296 ngoại lệ
d_44: 7093 ngoại lệ
d_45: 6358 ngoại lệ
d_46: 6302 ngoại lệ
d_47: 6299 ngoại lệ
d_48: 6042 ngoại lệ
d_49: 6507 ngoại lệ
d_50: 7168 ngoại lệ
d_51: 698